In [6]:
import pandas as pd
import numpy as np
import requests
import json
import time
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import random
import urllib.parse
import urllib.request
from citipy import citipy

In [10]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=500)
lat_lngs = zip(lats, lngs)

In [11]:
# Initialize empty lists for holding city, latitude, and longitude data
cities = []
latitudes = []
longitudes = []


In [12]:
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    # Use the geopy library to reverse geocode the lat_lng coordinates to get the closest city
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.reverse(f"{lat_lng[0]}, {lat_lng[1]}")
    if location is None:
        continue
    if 'city' in location.raw['address']:
        city = location.raw['address']['city']
    elif 'town' in location.raw['address']:
        #city = location.raw['address']['town']
        continue
    elif 'village' in location.raw['address']:
        #city = location.raw['address']['village']
        continue
    elif 'hamlet' in location.raw['address']:
       #city = location.raw['address']['hamlet']
       continue
    else:
        #city = 'Unknown'
        continue
    country = location.raw['address']['country']
    latitude = location.latitude
    longitude = location.longitude

    # Append the city, latitude, and longitude data to their respective lists
    cities.append(city + ', ' + country)
    latitudes.append(latitude)
    longitudes.append(longitude)

    # Pause for a short time to avoid making too many requests too quickly
    time.sleep(10)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=-82.63899675420625&lon=62.79594799578783&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
# Create a dataframe from the city, latitude, and longitude data
city_loc = {'Geopy City': cities, 'Latitude': latitudes, 'Longitude': longitudes}
city_loc_df = pd.DataFrame(city_loc)
city_loc_df.head()

,Geopy City,Latitude,Longitude
0,"Unorganized Division No. 22, Canada",54.819209,-96.115030
1,"Berry, Canada",48.843940,-78.315152
2,"ǁKhara Hais Local Municipality, South Africa",-27.688650,21.239166
3,"百色市, 中国",24.876100,105.367000


In [ ]:
# Create an empty list to store city names
city_citipy_loc = []

# Iterate over the rows in the dataframe
for index, row in city_loc_df.iterrows():
    
    # Use citipy to get the nearest city based on latitude and longitude
    city_cpy = citipy.nearest_city(row['Latitude'], row['Longitude']).city_name
    
    # Add the city name to the list
    city_citipy_loc.append(city_cpy)
    
# Add the list of city names as a new column in the dataframe
city_loc_df['Citipy City'] = city_citipy_loc
city_loc_df.head()

,Geopy City,Latitude,Longitude,Citipy City
0,"Unorganized Division No. 22, Canada",54.819209,-96.115030,thompson
1,"Berry, Canada",48.843940,-78.315152,amos
2,"ǁKhara Hais Local Municipality, South Africa",-27.688650,21.239166,upington
3,"百色市, 中国",24.876100,105.367000,anshun


In [ ]:
# Split the "Geopy City" column by comma and create two new columns
city_loc_df[['Geopy City', 'Geopy Country']] = city_loc_df['Geopy City'].str.split(',', expand=True)
city_loc_df

,Geopy City,Latitude,Longitude,Citipy City,Geopy Country
0,Unorganized Division No. 22,54.819209,-96.115030,thompson,Canada
1,Berry,48.843940,-78.315152,amos,Canada
2,ǁKhara Hais Local Municipality,-27.688650,21.239166,upington,South Africa
3,百色市,24.876100,105.367000,anshun,中国


In [ ]:
city_loc_df.to_csv("../comparisoncities.csv")